In [146]:
import pandas as pd
import joblib

path = '/Users/benjavitale/Documents/ML/TP_F/alquiler_AMBA_test.csv'
df_original = pd.read_csv(path, low_memory=False)
X_test = df_original
scaler = joblib.load('scaler.pkl')
mean_encoding = joblib.load('mean.pkl')
mean_m2 = joblib.load('m2.pkl')
media_stotalm2 = joblib.load('media_stotalm2.pkl')
nn_model = joblib.load('nn_model.pkl')
best_gbr_model = joblib.load('best_gbr_model.pkl')
linear_model = joblib.load('linear_model.pkl')
X_train = joblib.load('X_train.pkl')
mean_m2_tot = joblib.load('mean_m2_tot.pkl')
mean_precio = joblib.load('mean_precio.pkl')
X_test.head()

,id,id_grid,MesListing,TIPOPROPIEDAD,STotalM2,SConstrM2,Dormitorios,Banos,Ambientes,SitioOrigen,...,AreaParrillas,CanchaTennis,AreaCine,ITE_ADD_CITY_NAME,ITE_ADD_STATE_NAME,ITE_ADD_NEIGHBORHOOD_NAME,ITE_TIPO_PROD,LONGITUDE,LATITUDE,year
0,1,57427,2022-09-01,Departamento,38.0,38.0,1,1,2,NaN,...,0.0,NaN,NaN,Capital Federal,Capital Federal,Villa del Parque,N,-58.481301,-34.611180,2022
1,2,53483,2021-10-01,Departamento,33.0,33.0,1,1,2,0,...,0.0,0,0,Capital Federal,Capital Federal,Floresta,U,-58.472960,-34.632921,2021
2,3,53858,2022-06-01,Departamento,78.0,78.0,2,2,3,NaN,...,NaN,NaN,NaN,Capital Federal,Capital Federal,Barracas,U,-58.372587,-34.632289,2022
3,4,73216,2022-12-01,Departamento,37.0,35.0,1,1,2,NaN,...,NaN,NaN,NaN,Vicente López,Bs.As. G.B.A. Norte,Florida,U,-58.486302,-34.524556,2022
4,5,68619,2022-07-01,Departamento,42.0,36.0,0,1,1,NaN,...,0.0,NaN,NaN,Capital Federal,Capital Federal,Núñez,U,-58.467139,-34.550035,2022


In [147]:
X_test.drop(columns=['LONGITUDE'], inplace=True)
X_test.drop(columns=['LATITUDE'], inplace=True)
X_test.drop(columns=['id_grid'], inplace=True)
X_test.drop(columns=['TIPOPROPIEDAD'], inplace=True)
X_test.drop(columns=['SitioOrigen'], inplace=True)
X_test.drop(columns=['Cisterna'], inplace=True)
X_test.drop(columns=['year'], inplace=True)


In [148]:
X_test.fillna(0, inplace=True)

X_test['MesListing'] = pd.to_datetime(X_test['MesListing'])
X_test['anio'] = X_test['MesListing'].dt.year
X_test['mes'] = X_test['MesListing'].dt.month
X_test['dia'] = X_test['MesListing'].dt.day
X_test = X_test.drop(columns=['MesListing']) 


X_test['STotalM2'] = X_test['STotalM2'].apply(lambda x: media_stotalm2 if pd.isna(x) or x == 0 else x)





X_test = X_test[X_test['Antiguedad'].notnull()]
X_test['Antiguedad'] = X_test['Antiguedad'].replace(r'[^0-9.]', '', regex=True)
X_test['Antiguedad'] = pd.to_numeric(X_test['Antiguedad'], errors='coerce').astype('Int64')


def categorizar_antiguedad(antiguedad):
    if antiguedad <= 5:
        return 1  
    elif 5 < antiguedad <= 15:
        return 2  
    elif 15 < antiguedad <= 40:
        return 3  
    elif 40 < antiguedad <= 80:
        return 4  
    else:
        return 5  

X_test['Antiguedad'] = X_test['Antiguedad'].apply(lambda x: categorizar_antiguedad(x) if pd.notnull(x) else None)


def pasar_binarios(x):
    X_test[x] = X_test[x].replace({
        '0.0': 0, 'No': 0, '0': 0, '   0': 0, 
        '1.0': 1, '1': 1, 'Sí': 1
    })
    X_test[x] = pd.to_numeric(X_test[x], errors='coerce')
    X_test[x] = X_test[x].fillna(0)
    X_test[x].value_counts()
x = ['AreaJuegosInfantiles','Chimenea','Ascensor','SalonFiestas','Seguridad','Pileta','Cocheras','PistaJogging','EstacionamientoVisitas','Lobby','AreaParrillas','CanchaTennis','AreaCine', 'LocalesComerciales', 'Amoblado','Jacuzzi', 'AccesoInternet','BusinessCenter', 'Gimnasio', 'Laundry', 'Calefaccion', 'SalonDeUsosMul', 'AireAC', 'Recepcion', 'Estacionamiento']
for i in x:
    pasar_binarios(i)

X_test['ITE_TIPO_PROD'] = X_test['ITE_TIPO_PROD'].map({'N': 1, 'U': 2, 'S': 0})
X_test = pd.get_dummies(X_test, columns=['ITE_ADD_STATE_NAME'], prefix='state')

threshold = 1000
city_counts = X_test['ITE_ADD_CITY_NAME'].value_counts()
X_test['ITE_ADD_CITY_NAME'] = X_test['ITE_ADD_CITY_NAME'].apply(
    lambda x: x if city_counts[x] >= threshold else 'Otros'
)
X_test = pd.get_dummies(X_test, columns=['ITE_ADD_CITY_NAME'], prefix='city')




In [149]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 2. Aplicar Mean Encoding basado en el conjunto de entrenamiento
# NOTA: Usamos los mapeos calculados previamente en el conjunto de entrenamiento
X_test['mean_precio'] = X_test['ITE_ADD_NEIGHBORHOOD_NAME'].map(mean_encoding)
X_test['mean_m2'] = X_test['ITE_ADD_NEIGHBORHOOD_NAME'].map(mean_m2)

X_test['mean_precio'].fillna(mean_precio, inplace=True)
X_test['mean_m2'].fillna(mean_m2_tot, inplace=True)
# Crear la nueva columna 'mean_p/m2'
X_test['mean_p/m2'] = X_test['mean_precio'] / X_test['mean_m2']




columnas_con_nan = X_test.columns[X_test.isnull().any()].tolist()

# Imprimir las columnas con NaN y el número de valores faltantes por columna
if columnas_con_nan:
    print("Columnas con valores NaN:")
    for columna in columnas_con_nan:
        print(f"{columna}: {X_test[columna].isnull().sum()} valores NaN")
else:
    print("No hay columnas con valores NaN en X_test.")


X_test.drop(columns=['mean_precio'], inplace=True)
X_test.drop(columns=['mean_m2'], inplace=True)
X_test.drop(columns=['ITE_ADD_NEIGHBORHOOD_NAME'], inplace=True)





No hay columnas con valores NaN en X_test.


In [150]:





X_test = X_test[X_test['SConstrM2'] <= X_test['STotalM2']]

X_test['SNoConstrM2'] = X_test['STotalM2'] - X_test['SConstrM2']
X_test['SConstrRatio'] = X_test['SConstrM2'] / X_test['STotalM2']
X_test['SNoConstrRatio'] = 1 - X_test['SConstrRatio']


x = ['AreaJuegosInfantiles', 'Chimenea', 'Ascensor', 'SalonFiestas', 'Seguridad', 
     'Pileta', 'Cocheras', 'PistaJogging', 'EstacionamientoVisitas', 'Lobby', 
     'AreaParrillas', 'CanchaTennis', 'AreaCine', 'LocalesComerciales', 'Amoblado', 
     'Jacuzzi', 'AccesoInternet', 'BusinessCenter', 'Gimnasio', 'Laundry', 
     'Calefaccion', 'SalonDeUsosMul', 'AireAC', 'Recepcion', 'Estacionamiento']

X_test['TieneServicios'] = X_test[x].any(axis=1).astype(int)



for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0  # Agregar columnas faltantes con ceros

# Eliminar las columnas adicionales que no están en X_train
X_test = X_test[X_train.columns]




In [151]:
# Identificar columnas con valores NaN


In [152]:




X_test_scaled = scaler.transform(X_test)
y_pred_test_red_neuronal = nn_model.predict(X_test_scaled).ravel()

y_pred_test_gradient_boosting = best_gbr_model.predict(X_test)

y_pred_test_linear_model = linear_model.predict(X_test_scaled)

944/944 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step


In [155]:
x = []
for i in range(len(y_pred_test_gradient_boosting)):
    x.append(i)
np.array(x)
y  = pd.DataFrame(x)
y['prediccion'] = y_pred_test_gradient_boosting
y.to_csv("Vitale_Lynch_Gradient_Boosting.csv", index=False)

In [ ]:
x = []
for i in range(len(y_pred_test_linear_model)):
    x.append(i)
np.array(x)
y  = pd.DataFrame(x)
y['prediccion'] = y_pred_test_linear_model
y.to_csv("Vitale_Lynch_Linear_Model.csv", index=False)

In [ ]:
x = []
for i in range(len(y_pred_test_red_neuronal)):
    x.append(i)
np.array(x)
y  = pd.DataFrame(x)
y['prediccion'] = y_pred_test_red_neuronal
y.to_csv("Vitale_Lynch_Neural_Network.csv", index=False)